# **Iteration 2- ALBERT** 
## MSc Project
## Surya L Ramesh (0206793)

## **Load libraries**

In [14]:

!pip install transformers
!pip install sentencepiece

In [15]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
from transformers import AlbertTokenizer
import numpy as np

import os
import pickle
import fnmatch
from termcolor import colored
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import torch
import torch.nn as nn
import random

import spacy
import shutil  # for high-level operations on files

## **Check GPU and memory availability** 
   - code is from Google Colab official examples: https://colab.research.google.com/notebooks/pro.ipynb

In [16]:
# check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Aug 29 14:41:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.78       Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   35C    P8     5W /  N/A |   1040MiB /  8192MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
# check Memory 

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 17.0 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
device_type = "cuda:0" if torch.cuda.is_available() else "cpu"

cpu


## **Mount  google drive and access datasets**

In [19]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Use Relative path 
#path = 'SNLI_Data_Csv'
path = "/content/drive/MyDrive/CleanedData_notEncoded"
modelPath = "/content/drive/MyDrive/CleanedData_notEncoded/model_19_07"
dirList = os.listdir(path)

In [ ]:

dirList

## **Load and view the previously cleaned dataset** 
-  after cleaning , only wanted columns are kept , remove'-' rows, from gold_label, remove rows with na , rename columns,

In [20]:
for dir in dirList:
  sub_path = os.path.join(path,dir) 
  #print(sub_path)
  if fnmatch.fnmatch(sub_path,'*df_train.csv*'):
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    train_tidy = pd.read_csv(sub_path,nrows =350000)  # changed to 10K for experimentation 
    
    
  if fnmatch.fnmatch(sub_path,'*df_val*'):  
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    val_tidy = pd.read_csv(sub_path,nrows =10000) 

  if fnmatch.fnmatch(sub_path,'*df_test.csv*'):  
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    test_tidy = pd.read_csv(sub_path,nrows =10000) 

  if fnmatch.fnmatch(sub_path,'*df_testlegal.csv*'):  
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    testlegal_tidy = pd.read_csv(sub_path,nrows =20) 




NameError: name 'dirList' is not defined

In [ ]:
test_tidy.head(5)

In [ ]:
train_tidy.head(5)

In [ ]:
val_tidy.head(5)

## **Define class for SNLIDataAlbert**

The class definition and training part of the code for ALBERT has been adapted from https://github.com/dh1105/Sentence-Entailment/blob/main/Sentence_Entailment_BERT.ipynb


In [21]:
''' initialising , data Loader (batches) ,encoding ,  Add [CLS] and [SEP] tokens, join the two sentences for the train and val datasets'''

class SNLIDataAlbert(Dataset):

  def __init__(self, train_df, val_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'

    # second version of ALBERT is used
    
    self.tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)

  def load_data(self, df):
    MAX_LEN = 512   
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['premise'].to_list()
    hypothesis_list = df['hypothesis'].to_list()
    label_list = df['target'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )
    return train_loader, val_loader

### Ensure data is of type string 

In [22]:
train_tidy['premise'] = train_tidy['premise'].astype(str)
train_tidy['hypothesis'] = train_tidy['hypothesis'].astype(str)

NameError: name 'train_tidy' is not defined

In [ ]:
val_tidy['premise'] = val_tidy['premise'].astype(str)
val_tidy['hypothesis'] = val_tidy['hypothesis'].astype(str)

In [ ]:
test_tidy['premise'] = test_tidy['premise'].astype(str)
test_tidy['hypothesis'] = test_tidy['hypothesis'].astype(str)

In [ ]:
test_tidy['premise']

In [ ]:
test_tidy['premise']

## **Train on trainset and validate using validation set** 

In [ ]:

snli_dataset = SNLIDataAlbert(train_tidy, val_tidy)

In [ ]:
train_loader, val_loader = snli_dataset.get_data_loaders(batch_size=16)

## **Define class for SNLIDataAlbertPredictor**

In [ ]:
''' This class is for the Test set (prediction) to Initialize , data Loader (batches) ,
encoding ,  Add [CLS] and [SEP] tokens, join the two sentences '''


class SNLIDataAlbertPredictor(Dataset):

  def __init__(self,input_df):
    self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

    self.input_df = input_df

    self.base_path = '/content/'
    # second version of ALBERT is used as earlier
    self.tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)
    self.input_data = None
   # self.val_data = None
    self.init_data()

  def init_data(self):
    
    self.input_data = self.load_data(self.input_df)

  def load_data(self, df):
    MAX_LEN = 512   
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['premise'].to_list()
    hypothesis_list = df['hypothesis'].to_list()
    label_list = df['target'].to_list()

    for (premise, hypothesis,label) in zip(premise_list, hypothesis_list,label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids,y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    input_loader = DataLoader(
      self.input_data,
      shuffle=shuffle,
      batch_size=batch_size
    )
    return input_loader

In [ ]:
# Adam optimizer and pretrained AlbertSequenceClassification

from transformers import AlbertForSequenceClassification, AdamW

model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=3)
model.to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
# code from huggingface example - https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L102

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
# save model 

def save_checkpoint(state, is_best, epoch ):
   path_to_model=modelPath + '/model_epoch{}.pt'.format( epoch+1)
   torch.save(state, path_to_model)
   print("The model has been saved in {}".format(path_to_model))
   if is_best:
        shutil.copyfile(path_to_model, 'model_best.pth.tar')

In [ ]:
import tensorflow as tf
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
import time

EPOCHS = 10 # maximum no.of epochs to run 

def train(model, train_loader, val_loader, optimizer,epoch_to_resume):  
  total_step = len(train_loader)
  best_acc = 0
  is_best = False

  # check to resume from a previously saved epoch checkpoint

  if (epoch_to_resume> 0 ):
        path_to_model= modelPath +'/model_epoch{}.pt'.format( epoch_to_resume)
        if os.path.isfile(path_to_model):
            print("=> loading checkpoint '{}'".format(path_to_model))
            checkpoint = torch.load(path_to_model)
            savd_epoch = checkpoint['epoch']
            best_acc = checkpoint['best_acc']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(path_to_model, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(path_to_model))

  # training and validation for each epoch

  for epoch in range(epoch_to_resume, EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0

    # for each batch 
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      
      result = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels)
      
      loss = result.loss
      prediction = result.logits

      # check if there are nan in prediction

      if(torch.any(prediction.isnan())):
        break;

      # call accuracy function 
      acc = multi_acc(prediction, labels)
   
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    # get average train accuracy and loss across all batches 
    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()#Surya find out
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        
        result = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels)
        loss = result.loss
        prediction = result.logits
        if(torch.any(prediction.isnan())):
          break;
        
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    if(best_acc< val_acc):
      best_acc = val_acc 
      is_best = True
    else:
      is_best = False
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    

    # Saving the model
    save_checkpoint({
            'epoch': epoch + 1,
            'best_acc': best_acc,
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
        }, is_best,epoch)
   

In [ ]:
# commented out once epoch is chosen 
'''- this section is for training the model on trainset and validating on validation set-commented out'''

#%%time
#epoch_to_resume = 6
#train(model, train_loader, val_loader, optimizer,epoch_to_resume)

In [ ]:
print('Done')

In [ ]:
import time

EPOCHS = 6 # maximum no.of epochs to run 

def test(model, test_loader, optimizer,epoch_to_use):  
  # check to resume from a previously saved epoch checkpoint
  if (epoch_to_use > 0 ):
        path_to_model= modelPath +'/model_epoch{}.pt'.format( epoch_to_resume)
        if os.path.isfile(path_to_model):
            print("=> loading checkpoint '{}'".format(path_to_model))
            checkpoint = torch.load(path_to_model)
            savd_epoch = checkpoint['epoch']
            best_acc = checkpoint['best_acc']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(path_to_model, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(path_to_model))
            return; 
  y_pred_all = np.array([])
  y_test_all = np.array([])
  start = time.time() 
  total_test_loss = 0
  total_test_acc  = 0

  for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(test_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      
      result = model(pair_token_ids, 
                            token_type_ids=seg_ids, 
                            attention_mask=mask_ids, 
                            labels=labels)
      loss = result.loss
      prediction = result.logits
      if(torch.any(prediction.isnan())):
        break;
      
      acc = multi_acc(prediction, labels)
      #Add all predictions to y_pred_arr and all lables to y_test_arr
      y_pred = torch.log_softmax(prediction, dim=1).argmax(dim=1)
      y_pred_arr =  y_pred.to("cpu").numpy().squeeze() 
      y_test_arr = labels.to("cpu").numpy().squeeze()
      y_pred_all = np.append(y_pred_all,y_pred_arr, axis=0)
      y_test_all = np.append(y_test_all,y_test_arr, axis=0)      
      total_test_loss += loss.item()
      total_test_acc  += acc.item()

      test_acc  = total_test_acc/len(test_loader)
      test_loss = total_test_loss/len(test_loader)
      end = time.time()
      hours, rem = divmod(end-start, 3600)
      minutes, seconds = divmod(rem, 60)

  print(f'With Epoch {epoch_to_use}: test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  return y_pred_all,y_test_all 



In [ ]:
#Create object of SNLIAlberPredictor
#GEt test dataloader
snli_testset = SNLIDataAlbertPredictor(test_tidy)
test_loader = snli_testset.get_data_loaders(batch_size=16)

In [ ]:
test_tidy

In [ ]:
#epoch_to_use = 6
epoch_to_resume = 6
y_pred_all,y_test_all = test(model, test_loader, optimizer,epoch_to_resume)

## **Predict on Testset and Visualize**

In [24]:
# Libraries for this section 

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import itertools

In [ ]:
''' function for custom confusion matrix in colour  '''

def plot_confusion_matrix_custom(cm, classes, title='Confusion matrix - ALBERT', cmap=plt.cm.Blues):
  
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes, rotation=45)

    fmt =  'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False) 
   
    plt.show()

In [ ]:
''' function written to print classification report and display custom confusion matrix'''



def visualize_confusion_matrix(y_pred_argmax, y_true):

    classes = ['entailment','contradiction','neutral']
    cm = confusion_matrix(y_true, y_pred_argmax)
    con_mat_df = pd.DataFrame(cm)
    
    print(classification_report(y_pred_argmax, y_true,target_names=classes))

    plot_confusion_matrix_custom(cm,classes)


In [ ]:
print(y_pred_all)
print(y_test_all)

In [ ]:
#classification report and Confusion matrix 
visualize_confusion_matrix(y_pred_all,y_test_all)

## ---------------------------------------------- **End of code for ALBERT** ------------------------------------------------------------------------

## **This section is for checking individual sentence-pair inputs from user**

In [ ]:
def check_similarity(sentence1, sentence2):
           
       data_input = {'premise':[sentence1], 'hypothesis':[sentence2]}
       df_input = pd.DataFrame(data_input, columns = ['premise','hypothesis'])
       
       input_dataset = SNLIDataAlbertPredictor(df_input)
       input_loader = input_dataset.get_data_loaders(batch_size=1)
       
       (pair_token_ids, mask_ids, seg_ids) = next(iter(input_loader))
       pair_token_ids = pair_token_ids.to(device)
       mask_ids = mask_ids.to(device)
       seg_ids = seg_ids.to(device)
       result = model(pair_token_ids, 
                                     token_type_ids=seg_ids, 
                                     attention_mask=mask_ids)
       prediction = result.logits #Predition in tensor Form
       softmax =torch.log_softmax(prediction, dim=1)
       pred =softmax.argmax(dim=1)
       #target_map = {0: 'contradiction',1:'entailment',2:'neutral'}# error 
       target_map = {0: 'entailment',1:'contradiction',2:'neutral'}
       #target_map = {0:'entailment': 0, 'contradiction': 1, 'neutral': 2}
       
       if device_type == "cpu":
          outcome = target_map[pred.data.cpu().numpy()[0]]
       else:
           outcome = target_map[pred[0]]  
       return  outcome # modified to get value from tensor

In [ ]:
sentence1 ='This church choir sings to the masses as they sing joyous songs from the book at a church.'
sentence2 ='A choir singing at a baseball game.'
data_input = {'premise':[sentence1], 'hypothesis':[sentence2],'target':['entailment']}  #### 
df_input = pd.DataFrame(data_input, columns = ['premise','hypothesis','target'])

df_input.head()

In [ ]:
input_dataset = SNLIDataAlbertPredictor(df_input)
input_loader = input_dataset.get_data_loaders(batch_size=1)
#label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

## Testing with Legal based sentences provided by company

In [ ]:
# function to load data and tidy it 
# pass the dataframe, kepp only unwanted columns, remove'-' rows, from gold_label, remove rows with na , rename columns,lower case, replace all given special characters
# diplay head()

def data_load_tidy(dataframe): # remove isTest- not needed???
    
    dataframe = dataframe[['sentence1','sentence2','gold_label']]
    
    dataframe = dataframe[dataframe['gold_label'] != '-' ]
    dataframe= dataframe.dropna(axis=0, inplace=False) # fixed additional nan in test and val with inplace=False 
    data_tidy = dataframe.rename(columns= {'sentence1':'premise', 'sentence2':'hypothesis', 'gold_label':'target'}, inplace=False)
    #data_tidy['premise'] = data_tidy['premise'].astype(str)
  
    data_tidy['premise'] = data_tidy.premise.str.lower().str.replace('[^a-zA-Z ]' , '')  
    data_tidy['hypothesis'] = data_tidy.hypothesis.str.lower().str.replace('[^a-zA-Z ]' , '')    # lower, replace all  given special characters
  
    print(colored('\n  data is tidied ....','green')) 
    return data_tidy

    

In [ ]:
for dir in dirList:
  sub_path = os.path.join(path,dir) 
 
  if fnmatch.fnmatch(sub_path,'*df_testlegal.csv*'):  
    print(colored('\n In Filename and path: %s' % (sub_path),'blue')) 
    df_legaltest = pd.read_csv(sub_path,nrows =20) 
    legaltest_tidy = data_load_tidy(df_legaltest)

In [ ]:
df_legaltest.head(5)

In [ ]:
legaltest_tidy.head(3)

In [ ]:
legaltest_tidy.shape[0]

In [ ]:
#legalset = legaltest_tidy[['premise','hypothesis','target']]

In [ ]:
#Create object of SNLIAlberPredictor
#get legaltest set
snli_legaltestset = SNLIDataAlbertPredictor(legaltest_tidy)
legaltest_loader = snli_legaltestset.get_data_loaders(batch_size=16)

In [ ]:
# for ALBERT - testing on small Legalset
epoch_to_resume = 6
y_pred_legalall,y_test_legalall = test(model, legaltest_loader, optimizer,epoch_to_resume)

## Download colab notebook as HTML

In [ ]:
## download as html
%%shell
jupyter nbconvert --to html /content/Iteration2_ALBERT.ipynb

